### 🌍 Environment: LunarLander-v2

Mediul **`LunarLander-v2`** face parte din biblioteca `gymnasium` și simulează o navetă spațială care trebuie să aterizeze controlat între două steaguri.  
Scopul agentului este să învețe să folosească motoarele pentru a-și stabiliza poziția, unghiul și viteza, astfel încât să aterizeze vertical, fără impact violent.

- **Spațiu de stare**: 8 dimensiuni (poziție, viteză, unghi, contact picioare)
- **Spațiu de acțiuni**: 4 acțiuni discrete (`none`, `left engine`, `main engine`, `right engine`)
- **Recompensă**:  
  +100…140 pentru aterizare reușită,  
  penalizări pentru coliziuni, unghiuri mari și consum excesiv de combustibil.

În codul de mai jos:
- definim environmentul, parametrii de învățare și coeficienții Q-Learning (`α`, `γ`, `ε`);
- aplicăm **discretizare** pentru a transforma spațiul continuu într-un set finit de stări,  
  astfel încât algoritmul Q-Learning să poată funcționa eficient.


In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make("LunarLander-v2", render_mode=None)
alpha = 0.1
gamma = 0.99
epsilon, eps_min, eps_decay = 1.0, 0.01, 0.995
episodes = 1000

n_actions = env.action_space.n

# Discretizare: definim numărul de "bin-uri" pentru fiecare dimensiune
bins = [
    np.linspace(-1.5, 1.5, 7),  # x coord
    np.linspace(-1.5, 1.5, 7),  # y coord
    np.linspace(-2.0, 2.0, 7),  # x velocity
    np.linspace(-2.0, 2.0, 7),  # y velocity
    np.linspace(-np.pi, np.pi, 7),  # angle
    np.linspace(-5.0, 5.0, 7),  # angular velocity
    [0, 1],  # leg contact left
    [0, 1],  # leg contact right
]

def discretize(obs):
    state_idx = []
    for i, b in enumerate(bins):
        if len(b) == 2: 
            state_idx.append(int(obs[i]))
        else:
            state_idx.append(np.digitize(obs[i], b))
    return tuple(state_idx)



### 🤖 Q-Learning – Antrenarea agentului

În acest pas, agentul este antrenat folosind algoritmul **Q-Learning clasic**, care actualizează valorile Q pentru fiecare stare și acțiune.  
Agentul echilibrează **explorarea** (alegerea aleatorie de acțiuni) și **exploatarea** (folosirea celor mai bune acțiuni cunoscute) prin parametrul `ε` (*epsilon*).  

După fiecare episod:
- valorile Q sunt ajustate în funcție de recompensa primită și starea următoare;
- `ε` scade treptat, reducând explorarea în favoarea acțiunilor învățate;
- se urmărește evoluția **reward-ului mediu** pentru a evalua progresul agentului.


In [ ]:
Q = np.zeros([len(b)+1 for b in bins] + [n_actions])
rewards_per_episode = []

for ep in range(episodes):
    s, _ = env.reset()
    s_disc = discretize(s)
    done = False
    total = 0

    while not done:
        # Explorare / exploatare
        if np.random.rand() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s_disc])

        s2, r, done, truncated, _ = env.step(a)
        s2_disc = discretize(s2)

        # Actualizare Q
        Q[s_disc][a] += alpha * (r + gamma * np.max(Q[s2_disc]) * (not done) - Q[s_disc][a])
        s_disc = s2_disc
        total += r

    epsilon = max(eps_min, epsilon * eps_decay)
    rewards_per_episode.append(total)

    if (ep + 1) % 100 == 0:
        avg = np.mean(rewards_per_episode[-100:])
        print(f"Episod {ep+1}/{episodes} | Reward mediu: {avg:.2f} | ε={epsilon:.3f}")

window = 100
plt.plot(np.convolve(rewards_per_episode, np.ones(window)/window, mode='valid'))
plt.title("Evoluția reward-ului mediu - LunarLander Q-Learning")
plt.xlabel("Episoade")
plt.ylabel("Reward mediu (100 episoade)")
plt.grid(True)
plt.show()



In [ ]:
# Vizualizare după antrenare

env = gym.make("LunarLander-v2", render_mode="human")

def test_agent(Q, episodes=3):
    for ep in range(episodes):
        s, _ = env.reset()
        s_disc = discretize(s)
        done = False
        total = 0
        while not done:
            a = np.argmax(Q[s_disc])
            s, r, done, truncated, _ = env.step(a)
            s_disc = discretize(s)
            total += r
        print(f"Episod {ep+1} → Reward total: {total:.2f}")

test_agent(Q) 
env.close()


### 🧮 Dimensiunea spațiului de stări și impactul asupra Q-Table

În algoritmii de tip **Q-Learning**, fiecare combinație posibilă de stare și acțiune este stocată în tabelul Q.  
Prin urmare, dimensiunea spațiului de stări influențează direct **mărimea Q-table-ului** și memoria necesară pentru antrenare.

Codul de mai jos calculează și vizualizează cum crește dimensiunea Q-table-ului în funcție de numărul de **bin-uri** folosite pentru discretizarea fiecărei variabile de stare.  
Se observă o **creștere exponențială** — discretizări mai fine oferă precizie mai mare, dar cer mult mai multă memorie și timp de antrenare.


In [ ]:
import numpy as np

def q_table_size(bins_per_dim, n_actions):
    n_states = np.prod([b + 1 for b in bins_per_dim])
    q_size = n_states * n_actions
    mem_mb = q_size * 8 / (1024**2)
    return n_states, q_size, mem_mb

scenarios = {
    "3 bins": [3,3,3,3,3,3,2,2],
    "5 bins": [5,5,5,5,5,5,2,2],
    "7 bins": [7,7,7,7,7,7,2,2],
    "10 bins": [10,10,10,10,10,10,2,2],
    "15 bins": [15,15,15,15,15,15,2,2],
    "20 bins": [20,20,20,20,20,20,2,2],
    "50 bins": [50,50,50,50,50,50,2,2]
}

for name, bins in scenarios.items():
    n_states, q_size, mem = q_table_size(bins, 4)
    print(f"{name}: {n_states:,} stări | {q_size:,} Q-valori | {mem:.2f} MB")


import matplotlib.pyplot as plt

bins_range = [3, 5, 7, 10, 15, 20]
mem_usage = []

for b in bins_range:
    n_states, q_size, mem = q_table_size([b]*6 + [2,2], 4)
    mem_usage.append(mem)

plt.plot(bins_range, mem_usage, marker='o')
plt.title("Creșterea memoriei Q-Table în funcție de granularitate (LunarLander)")
plt.xlabel("Număr de bin-uri / dimensiune (primele 6 variabile)")
plt.ylabel("Dimensiune Q-table (MB)")
plt.grid(True)
plt.show()


# OPTIMIZARI! 
# IMBUNATATIRI!

### 🎯 Discretizare optimizată a spațiului de stare

Pentru a aplica Q-Learning pe un mediu cu stări continue, cum este **LunarLander-v2**, este necesară o transformare discretă a observațiilor.  
În acest exemplu:

- sunt păstrate doar cele **6 variabile esențiale** (poziție, viteză și unghi);  
- valorile sunt **normalizate** în intervalul `[-1, 1]` pentru coerență între dimensiuni;  
- se aplică o **discretizare neuniformă** cu mai multe bin-uri în zona de aterizare,  
  pentru a crește precizia în regiunile critice ale mediului.

Această metodă reduce dimensiunea Q-table-ului și accelerează învățarea fără pierderi semnificative de performanță.


In [ ]:
import numpy as np
import gymnasium as gym

# Inițializăm environmentul LunarLander
env = gym.make("LunarLander-v2")

# Selectăm doar cele 6 variabile relevante din cele 8 disponibile:
# [x, y, x_dot, y_dot, angle, angular_velocity]
def select_features(obs):
    return np.array(obs[:6])

# Intervalele aproximative pentru fiecare variabilă (pentru normalizare)
state_bounds = np.array([
    [-1.5, 1.5],    # poziție orizontală
    [-1.5, 1.5],    # poziție verticală
    [-2.0, 2.0],    # viteză orizontală
    [-2.0, 2.0],    # viteză verticală
    [-np.pi, np.pi],# unghi
    [-5.0, 5.0]     # viteză unghiulară
])

# Normalizăm starea în intervalul [-1, 1] pentru o scalare uniformă
def normalize_state(state):
    return 2 * (state - state_bounds[:,0]) / (state_bounds[:,1] - state_bounds[:,0]) - 1

# Definim bin-urile pentru discretizare
# y (poziția verticală) are bin-uri mai dense aproape de sol (unde are loc aterizarea)
bins = [
    np.linspace(-1.5, 1.5, 10),  
    np.concatenate([np.linspace(-1.5, 0, 20), np.linspace(0, 1.5, 10)]),  
    np.linspace(-2.0, 2.0, 8),   
    np.linspace(-2.0, 2.0, 8),   
    np.linspace(-np.pi, np.pi, 12),  
    np.linspace(-5.0, 5.0, 10)   
]

# Convertim observația continuă într-un tuple de indici discreți
def discretize(obs):
    features = select_features(obs)
    features = normalize_state(features)
    state_idx = [np.digitize(features[i], bins[i]) for i in range(len(bins))]
    return tuple(state_idx)

# Testăm discretizarea pentru o observație nouă
obs, _ = env.reset()
disc_state = discretize(obs)

In [ ]:
alpha, gamma = 0.1, 0.99
epsilon, eps_min, eps_decay = 1.0, 0.05, 0.995
episodes = 2000
n_actions = env.action_space.n

Q = np.zeros([len(b)+1 for b in bins] + [n_actions])
rewards = []

for ep in range(episodes):
    obs, _ = env.reset()
    s = discretize(obs)
    done = False
    total = 0

    while not done:
        if np.random.rand() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s])
        obs2, r, done, truncated, _ = env.step(a)
        s2 = discretize(obs2)
        Q[s][a] += alpha * (r + gamma * np.max(Q[s2]) * (not done) - Q[s][a])
        s = s2
        total += r

    epsilon = max(eps_min, epsilon * eps_decay)
    rewards.append(total)

import matplotlib.pyplot as plt
window = 100
plt.figure(figsize=(8,4))
plt.plot(np.convolve(rewards, np.ones(window)/window, mode='valid'))
plt.title("Evoluția reward-ului mediu - Q-Learning LunarLander")
plt.xlabel("Episoade")
plt.ylabel("Reward mediu (100 episoade)")
plt.grid(True)
plt.show()


### 🎚️ Epsilon Decay Adaptiv

În loc ca rata de explorare `ε` să scadă constant, se ajustează dinamic în funcție de performanța agentului:  
- dacă **reward-ul mediu crește**, `ε` scade mai rapid → agentul explorează mai puțin și exploatează mai mult;  
- dacă **performanța stagnează sau scade**, `ε` crește ușor → agentul revine la explorare.  

Astfel, agentul își adaptează comportamentul în timp, echilibrând mai bine explorarea și exploatarea.


In [ ]:
import numpy as np
import gymnasium as gym

# Inițializăm environmentul LunarLander
env = gym.make("LunarLander-v2")

alpha, gamma = 0.1, 0.99
epsilon, eps_min, eps_decay = 1.0, 0.05, 0.995
episodes = 2000
n_actions = env.action_space.n

Q = np.zeros([len(b)+1 for b in bins] + [n_actions])
rewards = []
eps_history = []

for ep in range(episodes):
    obs, _ = env.reset()
    s = discretize(obs)
    done = False
    total = 0

    while not done:
        if np.random.rand() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s])
        obs2, r, done, truncated, _ = env.step(a)
        s2 = discretize(obs2)
        Q[s][a] += alpha * (r + gamma * np.max(Q[s2]) * (not done) - Q[s][a])
        s = s2
        total += r

    rewards.append(total)
    eps_history.append(epsilon)

    if ep > 100:
        avg_recent = np.mean(rewards[-100:])
        avg_past = np.mean(rewards[-200:-100]) if ep >= 200 else -np.inf
        if avg_recent > avg_past + 10:
            epsilon *= 0.9
        elif avg_recent < avg_past - 5:
            epsilon = min(1.0, epsilon * 1.05)
        else:
            epsilon *= eps_decay
    epsilon = max(eps_min, epsilon)

import matplotlib.pyplot as plt
window = 100
plt.figure(figsize=(8,4))
plt.plot(np.convolve(rewards, np.ones(window)/window, mode='valid'))
plt.title("Evoluția reward-ului mediu - Q-Learning cu epsilon adaptiv")
plt.xlabel("Episoade")
plt.ylabel("Reward mediu (100 episoade)")
plt.grid(True)
plt.show()

plt.figure(figsize=(8,4))
plt.plot(eps_history)
plt.title("Evoluția dinamică a epsilon")
plt.xlabel("Episoade")
plt.ylabel("ε")
plt.grid(True)
plt.show()


### ⚙️ Reward Shaping

Pentru a accelera învățarea și a ghida agentul spre comportamente mai naturale,  
se modifică recompensa implicită adăugând **penalizări** și **bonusuri** suplimentare:

- penalizări pentru unghiuri mari și viteză orizontală excesivă aproape de sol;  
- bonusuri pentru stabilitate (unghi mic și mișcare controlată).  

Această tehnică, numită *reward shaping*, ajută agentul să învețe mai rapid manevre sigure și să evite comportamente instabile chiar înainte de convergența completă a politicii.


In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt

env = gym.make("LunarLander-v2")

alpha, gamma = 0.1, 0.99
epsilon, eps_min, eps_decay = 1.0, 0.05, 0.995
episodes = 1500
n_actions = env.action_space.n

# Simplificăm discretizarea pentru exemplu
bins = [np.linspace(-1.5, 1.5, 8),
        np.linspace(-1.5, 1.5, 8),
        np.linspace(-2.0, 2.0, 8),
        np.linspace(-2.0, 2.0, 8),
        np.linspace(-np.pi, np.pi, 8),
        np.linspace(-5.0, 5.0, 8)]

def discretize(obs):
    features = obs[:6]
    return tuple(np.digitize(features[i], bins[i]) for i in range(len(bins)))

# Funcția de reward shaping
def shape_reward(r, obs):
    angle, x_dot, y = obs[4], obs[2], obs[1]
    shaped_r = r

    # Aplicăm shaping doar când agentul e aproape de sol
    if y < 0.2:
        shaped_r -= 2 * abs(angle)
        shaped_r -= 1 * abs(x_dot)

        # Bonus pentru stabilitate
        if abs(angle) < 0.1 and abs(x_dot) < 0.1:
            shaped_r += 3

    return shaped_r

Q = np.zeros([len(b)+1 for b in bins] + [n_actions])
rewards = []

for ep in range(episodes):
    obs, _ = env.reset()
    s = discretize(obs)
    done = False
    total = 0

    while not done:
        if np.random.rand() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s])
        obs2, r, done, truncated, _ = env.step(a)
        r_shaped = shape_reward(r, obs2)
        s2 = discretize(obs2)
        Q[s][a] += alpha * (r_shaped + gamma * np.max(Q[s2]) * (not done) - Q[s][a])
        s = s2
        total += r_shaped

    epsilon = max(eps_min, epsilon * eps_decay)
    rewards.append(total)

window = 100
plt.figure(figsize=(8,4))
plt.plot(np.convolve(rewards, np.ones(window)/window, mode='valid'))
plt.title("Evoluția reward-ului mediu - Q-Learning cu Reward Shaping")
plt.xlabel("Episoade")
plt.ylabel("Reward mediu (100 episoade)")
plt.grid(True)
plt.show()
